In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.99-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 369.1 kB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.2/155.2 kB 753.6 kB/s eta 0:00:00a 0:00:01
  Using cached kiwisolver-1.4.5-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 1.9 MB/s eta 0:00:00a 0:00:01


# The implementation of FLERT model from FLAIR library
### Model uses pretrained NER head and custom processing pipeline
### Dates are converted to standard format with dateparser library
### The model can also be found in ner_models.py

In [1]:
import torch
from flair.data import Sentence
from flair.models import SequenceTagger
from scripts import load_test_data, test_n_samples, convert_to_standard_date


In [2]:
class FlairModel():
    def __init__(self):
        self.model = SequenceTagger.load("flair/ner-english-ontonotes-large")
        
    def extract_flight_details(self, user_request):
        sentence = Sentence(user_request)
        
        # predict NER tags
        self.model.predict(sentence)
        
        # Initializing base variables
        name = 'Unspecified'
        departure = 'Unspecified'
        destination = 'Unspecified'
        date = 'Unspecified'
        
        # Extract named entities and their labels
        for entity in sentence.get_spans('ner'):
            if entity.tag == 'PERSON':
                name = entity.text
            elif entity.tag in ['GPE', 'LOC']: 
                if departure == 'Unspecified':
                    if ' from ' in user_request[:entity.start_position]:
                        departure = entity.text
                    elif ' to ' in user_request[:entity.start_position]:
                        destination = entity.text
                    else: departure = entity.text
                elif destination == 'Unspecified':
                    destination = entity.text
                        
            elif entity.tag == 'DATE':
                date = entity.text
                if 'the ' in date:
                    date = date.replace('the ', '')
                if ' of ' in date:
                    date = date.replace(' of ', ' ')
                if 'around' in date:
                    date = date.replace('around', '')
                date = convert_to_standard_date(date)
                    
        return name.title(), departure.title(), destination.title(), date



In [3]:
model = FlairModel()

2023-11-30 12:19:44,067 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [4]:
user_requests, required_data = load_test_data()

In [5]:
test_n_samples(model, user_requests, required_data,n=5)

Name: Olivia Parker, True_Name: Olivia Parker
Departure: Barcelona, True_Departure: Barcelona
Destination: Amsterdam, True_Destination: Amsterdam
Date: 20-05-2024, True_Date: 20-05-2024
---------------------------------------------
Name: Henry Wright, True_Name: Henry Wright
Departure: Seoul, True_Departure: Seoul
Destination: Sydney, True_Destination: Sydney
Date: 08-09-2024, True_Date: 08-09-2024
---------------------------------------------
Name: Lily Johnson, True_Name: Lily Johnson
Departure: Rome, True_Departure: Rome
Destination: Paris, True_Destination: Paris
Date: 12-07-2024, True_Date: 12-07-2024
---------------------------------------------
Name: Lucas Thompson, True_Name: Lucas Thompson
Departure: New Delhi, True_Departure: New Delhi
Destination: Dubai, True_Destination: Dubai
Date: 05-10-2024, True_Date: 05-10-2024
---------------------------------------------
Name: Isabelle Brown, True_Name: Isabelle Brown
Departure: Tokyo, True_Departure: Tokyo
Destination: London, True_